# CNN迁移学习

## 环境导入

In [1]:
import os
## 导入 Inceptionv3 模型
from keras.applications.inception_v3 import InceptionV3, preprocess_input

# 导入建立神经网络的基本模块
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

from keras.optimizers import *
from keras.losses import categorical_crossentropy

# 导入数据增强模块
import cv2
from keras_preprocessing.image import ImageDataGenerator

# 超参数调节
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

# 可视化
# from keras.utils import plot_model
# from keras_visualizer import visualizer
# from IPython.display import Image, SVG, display
import datetime
from packaging import version

# 回调
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# 混合精度
# from tensorflow.keras import mixed_precision



2023-01-07 20:15:32.529953: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## 参数区

In [2]:
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_global_policy(policy)
# os.environ['TF_KERAS'] = '1'  # 必须使用tf.keras
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=1'  # 启用XLA

In [3]:
DATASET_PATH_ROOT = '/data/DataSets/TWITTER_IMG_SENT_2015/dataset/'
OUT_PATH_ROOT = '/data/Models/TWITTER_SENT_2015/'
OUT_LOG_PATH = OUT_PATH_ROOT + 'logs/'

TOTAL_EPOCH = 500
VAL_FREQ = 1

超参数

200次迭代后表现良好

/home/ubuntu/notebook/Models/TWITTER_SENT_2015/logs/hparam_tuning/20230107-065755/run-2

In [4]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([512, ]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.2]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
HP_METHOD = hp.HParam('train_method', hp.Discrete(['TransferLearning', 'FineTune']))
HP_L_RATE = hp.HParam('learning_rate', hp.Discrete([0.001, 0.0001, 0.00001]))

## 数据准备

In [5]:
# 训练集
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        # rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)

#验证集
val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        # rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)

# 测试集
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, )

# 数据输入
train_generator = train_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}train', target_size=(299, 299), batch_size=617)
val_generator = val_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}validation', target_size=(299, 299), batch_size=176)
test_generator = test_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}test', target_size=(299, 299), batch_size=89)

Found 617 images belonging to 2 classes.
Found 176 images belonging to 2 classes.
Found 89 images belonging to 2 classes.


## 迁移学习

In [6]:
# 输出日志
LOG_DIR = OUT_LOG_PATH + 'hparam_tuning/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '/'
os.environ['TENSORBOARD_BINARY'] = '/usr/local/miniconda3/envs/TensorFlow/bin/tensorboard'

# @formatter:off
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR} --port 6006 --bind_all
# @formatter:on

Launching TensorBoard...

In [7]:
def start_transfer_learning(run_dir, hparams):
    # 构建基础模型
    base_model = InceptionV3(weights='imagenet', include_top=False)  #去掉最后一层

    # 增加新的输出层
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # 添加全局平均池化层 将 MxNxC 的张量转换成 1xC 张量，C是通道数
    x = Dense(hparams[HP_NUM_UNITS], activation='relu')(x)  # 添加一个全连接层
    x = Dropout(hparams[HP_DROPOUT])(x)  # 添加一个隐藏层
    predictions = Dense(2, activation='softmax')(x)  # 自定义自己的分类器，这是一个2分类的分类器
    model = Model(inputs=base_model.input, outputs=predictions)  # 构建我们需要训练的完整模型

    # 锁层
    base_model.trainable = False

    # 学习率调整
    if hparams[HP_OPTIMIZER] == 'adam':
        optimizer = Adam(learning_rate=hparams[HP_L_RATE])
    elif hparams[HP_OPTIMIZER] == 'svd':
        optimizer = SGD(learning_rate=hparams[HP_L_RATE])
    else:
        raise Exception('No optimizer')

    # 编译模型
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    # model.summary()

    # 训练
    model.fit(train_generator,
              steps_per_epoch=1,  #800
              epochs=TOTAL_EPOCH,
              validation_data=val_generator,
              validation_steps=1,
              validation_freq=VAL_FREQ,  # 验证频率
              class_weight=None,  # 样本是均衡的
              callbacks=
              [TensorBoard(log_dir=run_dir, ),  #TensorBoardprofile_batch='1,10',
               hp.KerasCallback(run_dir, hparams),  #超参数
               # EarlyStopping(monitor='val_accuracy',patience=20,baseline=0.88,restore_best_weights=True, ),  # 早停选项
               # ReduceLROnPlateau(monitor='val_accuracy', ),  # 学习率衰减
               ModelCheckpoint(f'{run_dir}/Twitter2015_iv3_tl.h5', monitor='val_accuracy', save_best_only=True, save_freq='epoch')  #检查点
               ],
              workers=10,  # 线程
              )

    # 微调 mixed8 248
    # for layer in model.layers[:248]: layer.trainable = False
    # for layer in model.layers[248:]: layer.trainable = True
    #
    # # 编译模型
    # model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    # model.summary()
    #
    # # 训练
    # model.fit(train_generator,
    #           steps_per_epoch=1,  #800
    #           epochs=TOTAL_EPOCH,
    #           validation_data=val_generator,
    #           validation_steps=1,
    #           validation_freq=VAL_FREQ,  # 验证频率
    #           class_weight=None,  # 样本是均衡的
    #           callbacks=
    #           [TensorBoard(log_dir=run_dir, histogram_freq=1, profile_batch='500,520'),  #TensorBoard
    #            hp.KerasCallback(run_dir, hparams),  #超参数
    #            EarlyStopping(monitor='val_accuracy', patience=20),  # 早停选项
    #            ReduceLROnPlateau(monitor='val_accuracy', patience=20),  # 学习率衰减
    #            ModelCheckpoint(f'{run_dir}/Twitter2015_iv3_tl.h5', monitor='val_accuracy', save_best_only=True, save_freq='epoch')  #检查点
    #            ],
    #           workers=5,  # 线程
    #           )

    # 在测试集上评估
    # scores = model.evaluate_generator(test_generator)
    # print(scores)

    # return scores


## 超参数调整

In [8]:
# 超参数运行
def run_hparams():
    session_num = 0
    for num_units in HP_NUM_UNITS.domain.values:
        for dropout_rate in HP_DROPOUT.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                for learning_rate in HP_L_RATE.domain.values:
                    start_transfer_learning(
                            LOG_DIR + f'run-{session_num}',
                            {
                                    HP_NUM_UNITS: num_units,
                                    HP_DROPOUT  : dropout_rate,
                                    HP_OPTIMIZER: optimizer,
                                    HP_L_RATE   : learning_rate,
                            }
                    )
                    session_num += 1

# run_hparams()

## 推理任务

In [9]:

# BEST_TRAINACC_MODEL = f'/home/ubuntu/notebook/Models/TWITTER_SENT_2015/logs/hparam_tuning/20230107-065755/run-7/Twitter2015_iv3_tl.h5'
# read_model = load_model(BEST_TRAINACC_MODEL)
# read_model.evaluate(test_generator)


In [10]:
# 2 0.7977527976036072
#4 0.8089
#6 0.82
#